# Homework I

Joana Peixinho (ist1103335) & Miguel Fernandes (ist1103573)


## I. Pen and Paper [13v]

Consider the following dataset


![Decision Tree](./decision_tree.png)


### 1) Consider 𝐱1–𝐱7 to be training observations, 𝐱8–𝐱9 to be testing observations, 𝑦1– 𝑦5 to be input variables and 𝑦6 to be the target variable. Hint: you can use scipy.stats.multivariate_normal for multivariate distribution calculus

### a.) [3.5v] Learn a Bayesian classifier assuming: i) {𝑦1, 𝑦2}, {𝑦3, 𝑦4} and {𝑦5} sets of independent variables (e.g., 𝑦1 ⫫ 𝑦3 yet 𝑦1 ⫫ 𝑦2), and  ii) 𝑦1 × 𝑦2 ∈ ℝ2 is normally distributed.  Show all parameters (distributions and priors for subsequent testing).



### b.) [2.5v] Under a MAP assumption, classify each testing observation showing all your calculus.


### c.) [2v] Consider that the default decision threshold of 𝜃 = 0.5 can be adjusted according to 𝑓(𝐱|𝜃) = { A 𝑃(A|𝐱) > 𝜃 B otherwise . Under a maximum likelihood assumption, what thresholds optimize testing accuracy?

### 2) Let 𝑦1 be the target numeric variable, 𝑦2-𝑦6 be the input variables where 𝑦2 is binarized under an equal-width (equal-range) discretization. For the evaluation of regressors, consider a 3-fold cross-validation over the full dataset (𝐱1- 𝐱9) without shuffling the observations.

a. [1v] Identify the observations and features per data fold after the binarization procedure.

b. [4v] Consider a distance-weighted 𝑘NN with 𝑘 = 3, Hamming distance (𝑑), and 1/𝑑 weighting. Compute the MAE of this 𝑘NN regressor for the 1st iteration of the cross-validation (i.e. train observations have the lower indices).


## Programming [7v]

**Considering the column_diagnosis.arff dataset available at the course webpage’s homework tab.
Using sklearn, apply a 10-fold stratified cross-validation with shuffling (random_state=0) for the
assessment of predictive models along this section.**


### 1) [3v]

**Compare the performance of 𝑘NN with 𝑘 = 5 and naïve Bayes with Gaussian assumption
(consider all remaining parameters for each classifier as sklearn’s default):
a. Plot two boxplots with the fold accuracies for each classifier.
b. Using scipy, test the hypothesis “𝑘NN is statistically superior to naïve Bayes regarding
accuracy”, asserting whether is true**


### 2) [2.5v]

**Consider two 𝑘NN predictors with 𝑘 = 1 and 𝑘 = 5 (uniform weights, Euclidean distance,
all remaining parameters as default). Plot the differences between the two cumulative confusion
matrices of the predictors. Comment.**


### 3) [1.5v]

**Considering the unique properties of column_diagnosis, identify three possible difficulties
of naïve Bayes when learning from the given dataset.**
